<h1>Aplicatie pentru a detecta daca un mail este spam sau nu</h1>

Pentru implementarea acesteia vom folosi 2 metode. 
Clasificator binar folosind CMMP.
Clasificator folosind Support Vector Machine.

Scopul aplicatiei este de a realiza o comparatie intre cele 2 si sa observam care dintre ele este cel mai probabil sa isi dea seama de tipul mail-ului.

In [37]:
# Importarea bibliotecilor pe care le vom folosi
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
# Citirea fisierului cu mail-uri de intrare
data = pd.read_csv('spam.csv', encoding = 'latin-1')

# Eliminarea coloanelor de care nu este nevoie in fisier
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)

# Redenumirea coloanelor pentru a le putea folosi mai usor in label si message
data.columns = ['label', 'message']

# Inlocuirea lui ham cu 1 si spam cu -1
data['label'] = data['label'].map({'ham': 1, 'spam': -1})

messages = data['message']
messages = messages.to_list()
labels = data['label']

<h1>Preprocesarea mesajelor</h1>
<p>In aceasta etapa se vor elimina toate semnele de punctuatie din mesaje si toate cuvintele vor fi transformate in lowercase.</p>

In [39]:
# Eliminarea punctuatiei din fiecare mesaj
for i in range(len(messages)):
    messages[i] = messages[i].lower()
    messages[i] = re.sub(r'\W', ' ', messages[i])
    messages[i] = re.sub(r'\s+', ' ', messages[i])


In [40]:
# Bag of words

countVec = CountVectorizer(ngram_range=(1, 1), stop_words = 'english')
countData = countVec.fit_transform(messages)

# Creez dataframe
dataFrame = pd.DataFrame(countData.toarray(), columns = countVec.get_feature_names_out())
print(dataFrame)

      00  000  000pes  008704050406  0089  0121  01223585236  01223585334  \
0      0    0       0             0     0     0            0            0   
1      0    0       0             0     0     0            0            0   
2      0    0       0             0     0     0            0            0   
3      0    0       0             0     0     0            0            0   
4      0    0       0             0     0     0            0            0   
...   ..  ...     ...           ...   ...   ...          ...          ...   
5567   0    0       0             0     0     0            0            0   
5568   0    0       0             0     0     0            0            0   
5569   0    0       0             0     0     0            0            0   
5570   0    0       0             0     0     0            0            0   
5571   0    0       0             0     0     0            0            0   

      0125698789  02  ...  ó_  û_  û_thanks  ûªm  ûªt  ûªve  ûï  ûïharry  û

<h3>Rezolvam sistemul de tip Ax = b folosind CMMP.</h3>

In [41]:
# Rezolvarea folosind functia din numpy
data = countData.toarray()
n, m = data.shape # n = numarul de linii, m = numarul de coloane

# adaug o coloana plina de 1 la finalul matricei
data = np.append(data, np.ones((n, 1)), axis = 1)
leastSquareSol = np.linalg.lstsq(data, labels, rcond = None)[0]

<h3>Etapa de testare</h3>
<p>In aceasta etapa vom folosi restul mailurilor ramase in datele citite si vom alege din ele pentru a testa daca mail-ul este spam sau ham.</p>

In [67]:
testData = messages[2786 : ]

id =  int(input("Introduceti o valoare a unui mesaj din vector: "))
print(testData[id])

u ûªve bin awarded å 50 to play 4 instant cash call 08715203028 to claim every 9th player wins min å 50 å 500 optout 08718727870


<h3>Testarea clasificatorului</h3> 
<p>Aceasta etapa va prezice testLabel iar in functie de valoarea acestuia, vom determina daca un mesaj este spam sau ham.<p>
<p>Daca testLabel o sa fie -1 atunci mesajul este spam, daca este 1 atunci este ham<p>   

In [68]:
# Transformarea mesajului intr-un vector de numere
sentence = testData[id]
sentence = sentence.lower()
sentence = re.sub(r'\W', ' ', sentence)
sentence = re.sub(r'\s+', ' ', sentence)

# Transformarea mesajului intr-un vector de numere
sentence = countVec.transform([sentence])
sentence = sentence.toarray()

# Predict the label
sentence = np.append(sentence, np.ones((1, 1)), axis = 1)
prediction = np.dot(sentence, leastSquareSol)

if (prediction > 0):
    print("Mesajul " + testData[id] + " nu este spam")
else:
    print("Mesajul este spam")


Mesajul este spam
